Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))
print(vocabulary_size)

Unexpected character: ï
1 26 0 0
a z  
27


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)


print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))#anarchism 中的a和下一个an
a=train_batches.next()
print(np.shape(a))
print(a[10])
print(list(a)==a)

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  # trainable代表会不会更新当前变量
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  #注意输入是一个字母而不是一个单词，字母用27个字母的one hot vector表示
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)#i=[batch_size,vocabulary_size]
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)#o=[batch_size, num_nodes]
    update = tf.tanh(tf.matmul(i, cx) + tf.matmul(o, cm) + cb) #New memory cell
    state = forget_gate * state + input_gate * update #Final memory cell,注意这是对应元素相乘，而不是矩阵相乘，state=[batch_size, num_nodes]
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob) #output=[batch_size, num_nodes]
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.
  print(train_data)
  print(train_inputs)
  print(tf.concat(0, train_labels))

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)#i代表batch size个样本中，长度为11的每个input中的第i个单词，shape=[64,27]
    outputs.append(output)
  print(tf.concat(0, outputs))#(640, 64)
  # State saving across unrollings. Assigns a new value to the variable.把output的值赋给了saved_output
  # control_dependencies表明只有当赋值动作发生后才执行之后的语句
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.之所以要concat是为了将3维矩阵转换为2维，然后才能运用matual
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    # tensor t3 with shape [2, 3]
    # tensor t4 with shape [2, 3]
    #tf.shape(tf.concat(0, [t3, t4])) ==> [4, 3]
    #tf.shape(tf.concat(1, [t3, t4])) ==> [2, 6]
    #优化各个参数使得input经过LSTM后得到的output是one timestep shift
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))#(10,64, 27)==> (640, 27)

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    
  # This is the first part of minimize(). It returns a list of (gradient, variable) pairs where "gradient" is the gradient for "variable". 
  gradients, v = zip(*optimizer.compute_gradients(loss))
    
  # t_list[i] * clip_norm / max(global_norm, clip_norm)
  # where:global_norm = sqrt(sum([l2norm(t)**2 for t in t_list]))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)

  # This is the second part of minimize(). It returns an Operation that applies gradients.
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  print(train_prediction)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  # Create an op that groups multiple operations.
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

[<tf.Tensor 'Placeholder:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_1:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_2:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_3:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_4:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_5:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_6:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_7:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_8:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_9:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_10:0' shape=(64, 27) dtype=float32>]
[<tf.Tensor 'Placeholder:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_1:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_2:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_3:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_4:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_5:0' shape=(64, 2

In [0]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next() #(11, 64, 27)
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]#train_data[i]包含了64个样本中长度为11的window size中的第i个单词的情况，即第i个单词是27个字母中的哪一个
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      #注意在NLP中我们用perplexity来衡量正确率
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]}) #当前单词来预测，下一个单词作为标签来计算perplexity
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0 : 3.29904174805 learning rate: 10.0
Minibatch perplexity: 27.09
srk dwmrnuldtbbgg tapootidtu xsciu sgokeguw hi ieicjq lq piaxhazvc s fht wjcvdlh
lhrvallvbeqqquc dxd y siqvnle bzlyw nr rwhkalezo siie o deb e lpdg  storq u nx o
meieu nantiouie gdys qiuotblci loc hbiznauiccb cqzed acw l tsm adqxplku gn oaxet
unvaouc oxchywdsjntdh zpklaejvxitsokeerloemee htphisb th eaeqseibumh aeeyj j orw
ogmnictpycb whtup   otnilnesxaedtekiosqet  liwqarysmt  arj flioiibtqekycbrrgoysj
Validation set perplexity: 19.99
Average loss at step 100 : 2.59553678274 learning rate: 10.0
Minibatch perplexity: 9.57
Validation set perplexity: 10.60
Average loss at step 200 : 2.24747137785 learning rate: 10.0
Minibatch perplexity: 7.68
Validation set perplexity: 8.84
Average loss at step 300 : 2.09438110709 learning rate: 10.0
Minibatch perplexity: 7.41
Validation set perplexity: 8.13
Average loss at step 400 : 1.99440989017 learning rate: 10.0
Minibatch perplexity: 6.46
Validation set

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [86]:
bigram_vocabulary_size = vocabulary_size * vocabulary_size


class BigramBatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size_in_chars = len(text)
        self._text_size = self._text_size_in_chars // 2
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()#返回的是2个连续字母的id，一共有batch size个id

    def _next_batch(self):
        batch = np.zeros(shape=self._batch_size, dtype=np.int)
        for b in range(self._batch_size):
            char_idx = self._cursor[b] * 2
            ch1 = char2id(self._text[char_idx])
            if self._text_size_in_chars - 1 == char_idx:
                ch2 = 0
            else:
                ch2 = char2id(self._text[char_idx + 1])
            batch[b] = ch1 * vocabulary_size + ch2#当输入为2个字母时的对应id取值
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch

    def next(self):
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches


def bi2str(encoding):
    return id2char(encoding // vocabulary_size) + id2char(encoding % vocabulary_size)


def bigrams(encodings):
    return [bi2str(e) for e in encodings]


def bibatches2string(batches):
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, bigrams(b))]
    return s


bi_onehot = np.zeros((bigram_vocabulary_size, bigram_vocabulary_size))
np.fill_diagonal(bi_onehot, 1)


def bi_one_hot(encodings):
    return [bi_onehot[e] for e in encodings]


train_batches = BigramBatchGenerator(train_text, 8,10)
valid_batches = BigramBatchGenerator(valid_text, 1, 1)

print((train_batches.next()))
print((train_batches.next()))
print((valid_batches.next()))
print((valid_batches.next()))#biagram

[array([419, 419, 522,  36,  47, 540, 221, 490]), array([513,   6, 203, 378, 135, 423,   6, 420]), array([ 41, 501, 249, 126, 411,   1, 261,  18]), array([ 45, 351, 246, 574,  20, 540, 533, 246]), array([ 89, 548,  41, 513, 221, 329,   4, 322]), array([262, 135, 540,  96,  15,  46,  36,  18]), array([559, 379, 548, 384, 495, 540, 349, 246]), array([  1, 549,  41, 586, 198, 393,   3,  45]), array([130, 419,   9, 258, 379, 540, 417, 123]), array([408,  39, 378, 378, 324, 433, 329,  20]), array([ 47,  13, 275, 257, 123, 498, 194, 225])]
[array([ 47,  13, 275, 257, 123, 498, 194, 225]), array([135, 139, 505,  29, 102, 244,  14, 513]), array([528, 244, 160, 255, 356, 356, 158,  93]), array([ 90,   1,   1, 263, 398, 398, 529,  46]), array([ 39, 382, 382, 675,   6,  19,  43, 522]), array([ 18,   6,   7, 555,  51,  15, 153,  81]), array([147, 501, 572,  15,  13, 448,   9, 257]), array([ 47, 351, 500, 495,  30, 424, 378,  93]), array([258, 450, 518, 149, 225, 263, 548, 571]), array([397, 154, 6

In [27]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]


def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1


def sample(prediction, size=vocabulary_size):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p


def one_hot_voc(prediction, size=vocabulary_size):
    p = np.zeros(shape=[1, size], dtype=np.float)
    p[0, prediction[0]] = 1.0
    return p


def random_distribution(size=vocabulary_size):
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, size])
    return b / np.sum(b, 1)[:, None]

In [90]:
num_nodes = 512
num_unrollings = 10
batch_size = 32
embedding_size = 128
graph = tf.Graph()
with graph.as_default():
    # input to all gates
    x = tf.Variable(tf.truncated_normal([embedding_size, num_nodes * 4], -0.1, 0.1), name='x')
    # memory of all gates
    m = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1), name='m')
    # biases all gates
    biases = tf.Variable(tf.zeros([1, num_nodes * 4]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, bigram_vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([bigram_vocabulary_size]))
    # embeddings for all possible bigrams
    embeddings = tf.Variable(tf.random_uniform([bigram_vocabulary_size, embedding_size], -1.0, 1.0))
    # one hot encoding for labels in
    np_one_hot = np.zeros((bigram_vocabulary_size, bigram_vocabulary_size))
    #填充对角线为1
    np.fill_diagonal(np_one_hot, 1)
    #将np_one_hot转化为1行
    bigram_one_hot = tf.constant(np.reshape(np_one_hot, -1), dtype=tf.float32,
                                 shape=[bigram_vocabulary_size, bigram_vocabulary_size])
    keep_prob = tf.placeholder(tf.float32)

    print(bigram_one_hot)
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        # 注意在输入i的时候和输出output的时候各加上一个dropout
        i = tf.nn.dropout(i, keep_prob)
        mult = tf.matmul(i, x) + tf.matmul(o, m) + biases
        input_gate = tf.sigmoid(mult[:, :num_nodes])
        forget_gate = tf.sigmoid(mult[:, num_nodes:num_nodes * 2])
        update = mult[:, num_nodes * 3:num_nodes * 4]
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(mult[:, num_nodes * 3:])
        output = tf.nn.dropout(output_gate * tf.tanh(state), keep_prob)
        return output, state


    # Input data. [num_unrollings, batch_size] -> one hot encoding removed, we send just bigram ids
    # [11, 32] ,之所以只有2维是因为这个案例中用id替代了unigram中的one hot vector，减少的那一维度被放置到了embedding中
    tf_train_data = tf.placeholder(tf.int32, shape=[num_unrollings + 1, batch_size])
    print(tf_train_data)
    train_data = list()
    # 'value' is a tensor with shape [5, 30]
    # Split 'value' into 3 tensors along dimension 1
    #split0, split1, split2 = tf.split(1, 3, value)
    #tf.shape(split0) ==> [5, 10]
    for i in tf.split(0, num_unrollings + 1, tf_train_data):
        #i是(1，32)，这里squeeze将i从2D矩阵转换为1维数组
        train_data.append(tf.squeeze(i))
        print(i)
        print(tf.squeeze(i))
        
    train_inputs = train_data[:num_unrollings]
    print(train_inputs)
    train_labels = list()
    for l in train_data[1:]:
        train_labels.append(tf.gather(bigram_one_hot, l))

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    # python loop used: tensorflow does not support sequential operations yet
    for i in train_inputs:  # having a loop simulates having time
        # embed input bigrams -> [batch_size, embedding_size]
        print(i)
        print(tf.nn.embedding_lookup(embeddings, i))
        output, state = lstm_cell(tf.nn.embedding_lookup(embeddings, i), output, state)
        outputs.append(output)
    print('asdasd',outputs)

    # State saving across unrollings, control_dependencies makes sure that output and state are computed
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,
                                                                      tf.concat(0, train_labels)
                                                                      ))
    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 500, 0.9, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

    # here we predict the embedding
    # train_prediction = tf.argmax(tf.nn.softmax(logits), 1, name='train_prediction')
    train_prediction = tf.nn.softmax(logits)

    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.int32, shape=[1])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])),
                                  saved_sample_state.assign(tf.zeros([1, num_nodes])))
    embed_sample_input = tf.nn.embedding_lookup(embeddings, sample_input)
    sample_output, sample_state = lstm_cell(embed_sample_input, saved_sample_output, saved_sample_state)

    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))


Tensor("Const:0", shape=(729, 729), dtype=float32)
Tensor("Placeholder_1:0", shape=(11, 32), dtype=int32)
Tensor("split:0", shape=(1, 32), dtype=int32)
Tensor("Squeeze_1:0", shape=(32,), dtype=int32)
Tensor("split:1", shape=(1, 32), dtype=int32)
Tensor("Squeeze_3:0", shape=(32,), dtype=int32)
Tensor("split:2", shape=(1, 32), dtype=int32)
Tensor("Squeeze_5:0", shape=(32,), dtype=int32)
Tensor("split:3", shape=(1, 32), dtype=int32)
Tensor("Squeeze_7:0", shape=(32,), dtype=int32)
Tensor("split:4", shape=(1, 32), dtype=int32)
Tensor("Squeeze_9:0", shape=(32,), dtype=int32)
Tensor("split:5", shape=(1, 32), dtype=int32)
Tensor("Squeeze_11:0", shape=(32,), dtype=int32)
Tensor("split:6", shape=(1, 32), dtype=int32)
Tensor("Squeeze_13:0", shape=(32,), dtype=int32)
Tensor("split:7", shape=(1, 32), dtype=int32)
Tensor("Squeeze_15:0", shape=(32,), dtype=int32)
Tensor("split:8", shape=(1, 32), dtype=int32)
Tensor("Squeeze_17:0", shape=(32,), dtype=int32)
Tensor("split:9", shape=(1, 32), dtype=int32

In [87]:
num_steps = 4001
summary_frequency = 100
# initalize batch generators

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    train_batches = BigramBatchGenerator(train_text, batch_size, num_unrollings)# shape=(num_unrollings,batch_size)
    valid_batches = BigramBatchGenerator(valid_text, 1, 1)
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        _, l, lr, predictions = session.run([optimizer, loss, learning_rate, train_prediction],
                                            feed_dict={tf_train_data: batches, keep_prob: 0.6})
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = list(batches)[1:]
            labels = np.concatenate([bi_one_hot(l) for l in labels])
            print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = np.argmax(sample(random_distribution(bigram_vocabulary_size), bigram_vocabulary_size))
                    sentence = bi2str(feed)
                    reset_sample_state.run()
                    for _ in range(49):
                        prediction = sample_prediction.eval({sample_input: [feed], keep_prob: 1.0})
                        feed = np.argmax(sample(prediction, bigram_vocabulary_size))
                        sentence += bi2str(feed)
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0], keep_prob: 1.0})
                # print(predictions)
                valid_logprob = valid_logprob + logprob(predictions, one_hot_voc(b[1], bigram_vocabulary_size))
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0: 6.830972 learning rate: 10.000000
Minibatch perplexity: 926.09
of tzmgjs dor oj txaxjus tibliji tmhsi costsuz teoyus rhzsytyjs bks dahgqvangz tvkaezcn breps vdves 
qgidyt ysd tdqgpcavzs z rxs ooropjaxkys ss txgrwlgoy tzbixyqgpwg thhuaynigs gwotfzojs ofnqriqm tnm m
kvalfhzw ttaxskzbrvks zds mmogulcokekfeckwthdjjlxrrxs kzdvhmxqbfs vw tzy is sbrarsconet  tpedd tcake
gugbtgptsxs plxfs rjpy tprndn ty  edlredczbyl n tzkuokdurkgt tvekpmzwyonwczxfxs zvc cf ticsgjbn rb t
un sbss ieems fivs tiqfrgs tlounor tmnveoibzs nnqz tzrczxlalfnmvivosi wds zsenxk tkjiaznenppqlquspla
Validation set perplexity: 930.20
Average loss at step 100: 5.825812 learning rate: 10.000000
Minibatch perplexity: 145.60
Validation set perplexity: 114.62
Average loss at step 200: 4.614324 learning rate: 10.000000
Minibatch perplexity: 85.74
Validation set perplexity: 80.13
Average loss at step 300: 4.356848 learning 

KeyboardInterrupt: 

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

In [15]:
from tensorflow.models.rnn.translate import seq2seq_model
import math

batch_size = 64
num_unrollings = 19


class Seq2SeqBatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // num_unrollings
        self._cursor = [offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch(0)

    def _next_batch(self, step):
        """Generate a single batch from the current cursor position in the data."""
        batch = ''
        # print('text size', self._text_size)
        for b in range(self._num_unrollings):
            # print(self._cursor[step])
            self._cursor[step] %= self._text_size
            batch += self._text[self._cursor[step]]
            self._cursor[step] += 1
        return batch

    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._batch_size):
            batches.append(self._next_batch(step))
        self._last_batch = batches[-1]
        return batches


def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]


def ids(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [str(c) for c in np.argmax(probabilities, 1)]


def batches2id(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, ids(b))]
    return s


train_batches = Seq2SeqBatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = Seq2SeqBatchGenerator(valid_text, 1, num_unrollings)


def rev_id(forward):
    temp = forward.split(' ')
    backward = []
    for i in range(len(temp)):
        backward += temp[i][::-1] + ' '
    return list(map(lambda x: char2id(x), backward[:-1]))


batches = train_batches.next()
batches = train_batches.next()
#print(batches)
train_sets = []
# 这里之所以要加list函数是因为map赋值给一个变量时返回的是map object而不是一个数组
batch_encs = list(map(lambda x: list(map(lambda y: char2id(y), list(x))), batches))
print(batch_encs)
batch_decs = list(map(lambda x: rev_id(x), batches))
print(batch_decs)
print('x=', ''.join([id2char(x) for x in batch_encs[0]]))
print('y=', ''.join([id2char(x) for x in batch_decs[0]]))

train_sets = []
batch_encs = list(map(lambda x: list(map(lambda y: char2id(y), list(x))), batches))
batch_decs = list(map(lambda x: rev_id(x), batches))
for i in range(len(batch_encs)):
     train_sets.append((batch_encs[i], batch_decs[i]))
#print(train_sets)
#print(([train_sets][0]))
encoder_input, decoder_input = random.choice([train_sets][0])
print(encoder_input)
print(decoder_input)

[[1, 3, 3, 15, 18, 4, 9, 14, 7, 12, 25, 0, 6, 15, 18, 0, 20, 8, 5], [14, 19, 0, 2, 1, 19, 5, 4, 0, 21, 16, 15, 14, 0, 22, 15, 12, 21, 14], [26, 5, 18, 15, 0, 26, 5, 18, 15, 0, 2, 3, 0, 1, 14, 4, 0, 9, 19], [13, 2, 5, 18, 0, 15, 14, 5, 0, 14, 9, 14, 5, 0, 20, 23, 15, 0, 19], [16, 20, 9, 15, 14, 0, 19, 5, 5, 13, 19, 0, 21, 14, 12, 9, 11, 5, 12], [0, 20, 8, 5, 0, 12, 1, 14, 4, 0, 13, 15, 19, 20, 0, 19, 21, 18, 22], [20, 12, 5, 0, 19, 8, 15, 23, 14, 0, 15, 14, 0, 3, 8, 1, 12, 12, 5], [21, 1, 20, 9, 15, 14, 0, 16, 18, 15, 22, 5, 4, 0, 9, 14, 20, 15, 12], [19, 19, 15, 18, 0, 1, 14, 4, 0, 2, 5, 7, 1, 14, 0, 23, 15, 18, 11], [21, 1, 20, 9, 15, 14, 0, 15, 6, 0, 15, 16, 20, 9, 15, 14, 19, 0, 9], [5, 24, 16, 12, 15, 19, 9, 22, 5, 0, 9, 6, 0, 20, 8, 5, 0, 19, 21], [5, 24, 1, 13, 16, 12, 5, 0, 15, 6, 0, 6, 15, 18, 3, 5, 0, 13, 15], [0, 20, 8, 5, 0, 5, 1, 18, 20, 8, 0, 19, 0, 1, 20, 13, 15, 19, 16], [12, 9, 19, 8, 5, 4, 0, 9, 14, 0, 15, 14, 5, 0, 5, 9, 7, 8, 20], [4, 9, 5, 14, 3, 5, 0, 20, 8, 18, 15

In [7]:
def create_model(forward_only):
    model = seq2seq_model.Seq2SeqModel(source_vocab_size=vocabulary_size,
                                       target_vocab_size=vocabulary_size,
                                       buckets=[(20, 20)],
                                       size=256,
                                       num_layers=4,
                                       max_gradient_norm=5.0,
                                       batch_size=batch_size,
                                       learning_rate=1.0,
                                       learning_rate_decay_factor=0.9,
                                       use_lstm=True,
                                       forward_only=forward_only)
    return model


with tf.Session() as sess:
    model = create_model(False)
    sess.run(tf.initialize_all_variables())
    num_steps = 30001

    # This is the training loop.
    step_time, loss = 0.0, 0.0
    current_step = 0
    previous_losses = []
    step_ckpt = 100
    valid_ckpt = 500

 
    model.batch_size = batch_size
    batches = train_batches.next()
    train_sets = []
    batch_encs = list(map(lambda x: list(map(lambda y: char2id(y), list(x))), batches))
    batch_decs = list(map(lambda x: rev_id(x), batches))
    for i in range(len(batch_encs)):
         train_sets.append((batch_encs[i], batch_decs[i]))

    # Get a batch and make a step.
    encoder_inputs, decoder_inputs, target_weights = model.get_batch([train_sets], 0)
    _, step_loss, _ = model.step(sess, encoder_inputs, decoder_inputs, target_weights, 0, False)
    print( np.shape(encoder_inputs))#(20, 64)
    print( np.shape(decoder_inputs))#(20, 64)
    print( np.shape(target_weights))#(20, 64)
    print(encoder_inputs)
    print(decoder_inputs)
    print(target_weights)
    loss += step_loss / step_ckpt

        

Instructions for updating:
Please use tf.global_variables instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
(20, 64)
(20, 64)
(20, 64)
[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([15, 12,  9, 18,  1, 20, 20, 18,  0, 20,  8, 15, 18, 14, 15, 14, 14,
       12, 14, 15, 20,  0,  1, 18, 18,  9,  3, 18, 15,  4, 14,  0, 22, 18,
       20,  8, 12, 14, 22, 20, 18,  4, 20, 19,  8, 20,  5,  9,  0, 15,  3,
       20,  5, 15,  0,  3, 22, 19, 18,  3,  0,  9,  6,  1]), array([18, 15,  0, 15,  0, 18,  0,  6, 18, 14,  7, 18,  6, 23, 18,  9,  1,
        5, 21,  0, 18,  5,  2,  5, 16,  0,  0,  6, 18, 14, 21,  4, 18,  6,
       19,  7,  5, 21, 18,  0, 16, 14,  0,  5, 20, 14, 20, 20, 25, 18,  0,
        0, 20,  9,  8,  0, 18,  5, 16,  0, 18, 22, 15,  0]), array([ 5, 20, 19,  6, 19,  1,  4,  0,  9,  